In [1]:

# Import Modules
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# Import the pitcher data
verlander_df = pd.read_csv(Path("../resources/verlander_update.csv"))
display(verlander_df.head())

# Drop extra columns
verlander_df = verlander_df.drop(columns=['player_name', 'batter', 'pitcher', 'des', 'home_team', 'away_team', 'hit_location',  'at_bat_number'])
verlander_df = verlander_df.drop(columns=['bb_type','pitch_name'])

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,stand,...,on_1b,outs_when_up,inning,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
0,SL,9/16/2022,"Verlander, Justin",669127,434378,strikeout,swinging_strike,9,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,5,Slider,0,2,Infield shift,Standard
1,FF,9/16/2022,"Verlander, Justin",669127,434378,NaN,foul,3,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,4,4-Seam Fastball,0,2,Infield shift,Standard
2,SL,9/16/2022,"Verlander, Justin",669127,434378,NaN,called_strike,13,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,3,Slider,0,2,Infield shift,Standard
3,FF,9/16/2022,"Verlander, Justin",669127,434378,NaN,foul,1,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,2,4-Seam Fastball,0,2,Infield shift,Standard
4,SL,9/16/2022,"Verlander, Justin",669127,434378,NaN,ball,8,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,1,Slider,0,2,Infield shift,Standard


In [3]:
display(verlander_df.tail())
verlander_df = verlander_df.iloc[::-1].reset_index(drop=True)
# verlander_df.iloc[-60:-30]
display(verlander_df.tail())
verlander_df.isna().sum()

,pitch_type,game_date,events,description,zone,stand,p_throws,type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
2331,FF,4/9/2022,strikeout,swinging_strike,8,L,R,S,2,2,NaN,NaN,NaN,0,1,5,0,0,Infield shift,Standard
2332,FF,4/9/2022,NaN,foul_tip,4,L,R,S,2,1,NaN,NaN,NaN,0,1,4,0,0,Infield shift,Standard
2333,FF,4/9/2022,NaN,ball,13,L,R,B,1,1,NaN,NaN,NaN,0,1,3,0,0,Infield shift,Standard
2334,FF,4/9/2022,NaN,called_strike,13,L,R,S,1,0,NaN,NaN,NaN,0,1,2,0,0,Infield shift,Standard
2335,FF,4/9/2022,NaN,ball,11,L,R,B,0,0,NaN,NaN,NaN,0,1,1,0,0,Infield shift,Standard


,pitch_type,game_date,events,description,zone,stand,p_throws,type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
2331,SL,9/16/2022,NaN,ball,8,R,R,B,0,0,NaN,NaN,NaN,2,5,1,0,2,Infield shift,Standard
2332,FF,9/16/2022,NaN,foul,1,R,R,S,1,0,NaN,NaN,NaN,2,5,2,0,2,Infield shift,Standard
2333,SL,9/16/2022,NaN,called_strike,13,R,R,S,1,1,NaN,NaN,NaN,2,5,3,0,2,Infield shift,Standard
2334,FF,9/16/2022,NaN,foul,3,R,R,S,1,2,NaN,NaN,NaN,2,5,4,0,2,Infield shift,Standard
2335,SL,9/16/2022,strikeout,swinging_strike,9,R,R,S,1,2,NaN,NaN,NaN,2,5,5,0,2,Infield shift,Standard


pitch_type                  0
game_date                   0
events                   1734
description                 0
zone                        0
stand                       0
p_throws                    0
type                        0
balls                       0
strikes                     0
on_3b                    2182
on_2b                    2058
on_1b                    1716
outs_when_up                0
inning                      0
pitch_number                0
bat_score                   0
fld_score                   0
if_fielding_alignment       0
of_fielding_alignment       0
dtype: int64

In [4]:
display(verlander_df[['on_3b', 'on_2b', 'on_1b']][2279:2281])


verlander_df.fillna(value = {"on_3b": 0, "on_2b": 0, "on_1b": 0} , inplace=True)

# Change player ID numbers for the on base dataframe to 1.0
verlander_df.on_1b[verlander_df.on_1b > 0] = 1.0
verlander_df.on_2b[verlander_df.on_2b > 0] = 1.0
verlander_df.on_3b[verlander_df.on_3b > 0] = 1.0

verlander_df[['on_3b', 'on_2b', 'on_1b']][2279:2281]

,on_3b,on_2b,on_1b
2279,NaN,NaN,657656.0
2280,NaN,NaN,657656.0


C:\anacondastuff\envs\dev\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\anacondastuff\envs\dev\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\anacondastuff\envs\dev\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,on_3b,on_2b,on_1b
2279,0.0,0.0,1.0
2280,0.0,0.0,1.0


In [5]:
# verlander_df['lagged_date']=verlander_df['game_date'].shift()
# verlander_df.head()

In [6]:
# verlander_df['date_change']=1
# verlander_df['date_change'][verlander_df[verlander_df['game_date']!=verlander_df['lagged_date']]]=0

In [7]:
display(verlander_df.isna().sum())
verlander_df.fillna(value = {"events": "nothing_major"} , inplace=True)
verlander_df.isna().sum()

pitch_type                  0
game_date                   0
events                   1734
description                 0
zone                        0
stand                       0
p_throws                    0
type                        0
balls                       0
strikes                     0
on_3b                       0
on_2b                       0
on_1b                       0
outs_when_up                0
inning                      0
pitch_number                0
bat_score                   0
fld_score                   0
if_fielding_alignment       0
of_fielding_alignment       0
dtype: int64

pitch_type               0
game_date                0
events                   0
description              0
zone                     0
stand                    0
p_throws                 0
type                     0
balls                    0
strikes                  0
on_3b                    0
on_2b                    0
on_1b                    0
outs_when_up             0
inning                   0
pitch_number             0
bat_score                0
fld_score                0
if_fielding_alignment    0
of_fielding_alignment    0
dtype: int64

In [8]:
verlander_df['zone'].value_counts()

14    419
11    313
12    298
6     200
5     176
2     164
9     148
3     139
8     119
13    116
4     104
1      99
7      41
Name: zone, dtype: int64

In [9]:
verlander_df['zone'] = verlander_df['zone'].map({1:'A', 2:'B',3:'C', 4:'D',5:'E', 6:'F',7:'G', 8:'H',9:'I', 10:'J',11:'K', 12:'L',13:'M', 14:'N'})

In [10]:
lagged_df = verlander_df.copy()[['zone',
                                 'type',
                                 'pitch_type',
                                 'description',
                                 'events',
                                ]].shift(1)

lagged_df = lagged_df.rename(columns = {
    'zone' : 'previous_zone',
    'type' : 'previous_type',
    'pitch_type' : 'previous_pitch_type',
    'description': 'previous_description',
    'events':'previous_event'
})

# display(lagged_df.tail(50))
display(lagged_df)

verlander_lagged_df = pd.concat([verlander_df, lagged_df], join='inner', axis = 1)

verlander_lagged_df = verlander_lagged_df.drop(columns = ['zone',
                                                          'type',
                                                          'description',
                                                          'events',
                                                         ])


# verlander_lagged_df.dropna(inplace=True)
# display(verlander_lagged_df.tail(30))
display(verlander_lagged_df)

display(verlander_lagged_df.isna().sum())

,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event
0,NaN,NaN,NaN,NaN,NaN
1,K,B,FF,ball,nothing_major
2,M,S,FF,called_strike,nothing_major
3,M,B,FF,ball,nothing_major
4,D,S,FF,foul_tip,nothing_major
...,...,...,...,...,...
2331,D,X,SL,hit_into_play,field_out
2332,H,B,SL,ball,nothing_major
2333,A,S,FF,foul,nothing_major
2334,M,S,SL,called_strike,nothing_major


,pitch_type,game_date,stand,p_throws,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,...,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event
0,FF,4/9/2022,L,R,0,0,0.0,0.0,0.0,0,...,1,0,0,Infield shift,Standard,NaN,NaN,NaN,NaN,NaN
1,FF,4/9/2022,L,R,1,0,0.0,0.0,0.0,0,...,2,0,0,Infield shift,Standard,K,B,FF,ball,nothing_major
2,FF,4/9/2022,L,R,1,1,0.0,0.0,0.0,0,...,3,0,0,Infield shift,Standard,M,S,FF,called_strike,nothing_major
3,FF,4/9/2022,L,R,2,1,0.0,0.0,0.0,0,...,4,0,0,Infield shift,Standard,M,B,FF,ball,nothing_major
4,FF,4/9/2022,L,R,2,2,0.0,0.0,0.0,0,...,5,0,0,Infield shift,Standard,D,S,FF,foul_tip,nothing_major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,SL,9/16/2022,R,R,0,0,0.0,0.0,0.0,2,...,1,0,2,Infield shift,Standard,D,X,SL,hit_into_play,field_out
2332,FF,9/16/2022,R,R,1,0,0.0,0.0,0.0,2,...,2,0,2,Infield shift,Standard,H,B,SL,ball,nothing_major
2333,SL,9/16/2022,R,R,1,1,0.0,0.0,0.0,2,...,3,0,2,Infield shift,Standard,A,S,FF,foul,nothing_major
2334,FF,9/16/2022,R,R,1,2,0.0,0.0,0.0,2,...,4,0,2,Infield shift,Standard,M,S,SL,called_strike,nothing_major


pitch_type               0
game_date                0
stand                    0
p_throws                 0
balls                    0
strikes                  0
on_3b                    0
on_2b                    0
on_1b                    0
outs_when_up             0
inning                   0
pitch_number             0
bat_score                0
fld_score                0
if_fielding_alignment    0
of_fielding_alignment    0
previous_zone            1
previous_type            1
previous_pitch_type      1
previous_description     1
previous_event           1
dtype: int64

## Feature Engineering

In [11]:
verlander_lagged_df.dropna(inplace=True)
verlander_lagged_df.reset_index(drop=True, inplace=True)
display(verlander_lagged_df.head())
verlander_lagged_df['ones'] = 1

pitch_count_df = verlander_lagged_df[['game_date', 'ones']]
pitch_count_df['pitch_count'] = pitch_count_df.groupby(['game_date']).cumcount()
pitch_count_df['pitch_count'] = pitch_count_df['pitch_count'] + 1

verlander_lagged_df.drop(columns=['ones'],inplace=True)
verlander_lagged_df = pd.concat([verlander_lagged_df, pitch_count_df], join='inner', axis = 1)
verlander_lagged_df.drop(columns=['ones'],inplace=True)
display(verlander_lagged_df.head(20))
verlander_lagged_df.tail(30)


,pitch_type,game_date,stand,p_throws,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,...,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event
0,FF,4/9/2022,L,R,1,0,0.0,0.0,0.0,0,...,2,0,0,Infield shift,Standard,K,B,FF,ball,nothing_major
1,FF,4/9/2022,L,R,1,1,0.0,0.0,0.0,0,...,3,0,0,Infield shift,Standard,M,S,FF,called_strike,nothing_major
2,FF,4/9/2022,L,R,2,1,0.0,0.0,0.0,0,...,4,0,0,Infield shift,Standard,M,B,FF,ball,nothing_major
3,FF,4/9/2022,L,R,2,2,0.0,0.0,0.0,0,...,5,0,0,Infield shift,Standard,D,S,FF,foul_tip,nothing_major
4,FF,4/9/2022,R,R,0,0,0.0,0.0,0.0,1,...,1,0,0,Infield shift,Standard,H,S,FF,swinging_strike,strikeout


C:\anacondastuff\envs\dev\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\anacondastuff\envs\dev\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,pitch_type,game_date,stand,p_throws,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,...,fld_score,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count
0,FF,4/9/2022,L,R,1,0,0.0,0.0,0.0,0,...,0,Infield shift,Standard,K,B,FF,ball,nothing_major,4/9/2022,1
1,FF,4/9/2022,L,R,1,1,0.0,0.0,0.0,0,...,0,Infield shift,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2
2,FF,4/9/2022,L,R,2,1,0.0,0.0,0.0,0,...,0,Infield shift,Standard,M,B,FF,ball,nothing_major,4/9/2022,3
3,FF,4/9/2022,L,R,2,2,0.0,0.0,0.0,0,...,0,Infield shift,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4
4,FF,4/9/2022,R,R,0,0,0.0,0.0,0.0,1,...,0,Infield shift,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5
5,FF,4/9/2022,R,R,1,0,0.0,0.0,0.0,1,...,0,Infield shift,Standard,L,B,FF,ball,nothing_major,4/9/2022,6
6,FF,4/9/2022,R,R,1,1,0.0,0.0,0.0,1,...,0,Infield shift,Standard,H,S,FF,foul,nothing_major,4/9/2022,7
7,SL,4/9/2022,R,R,1,2,0.0,0.0,0.0,1,...,0,Infield shift,Standard,C,S,FF,foul,nothing_major,4/9/2022,8
8,FF,4/9/2022,R,R,2,2,0.0,0.0,0.0,1,...,0,Infield shift,Standard,N,B,SL,ball,nothing_major,4/9/2022,9
9,FF,4/9/2022,R,R,0,0,0.0,0.0,0.0,2,...,0,Standard,Standard,B,X,FF,hit_into_play,field_out,4/9/2022,10


,pitch_type,game_date,stand,p_throws,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,...,fld_score,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count
2305,FF,9/16/2022,R,R,0,0,0.0,0.0,0.0,1,...,2,Infield shift,Standard,F,X,SL,hit_into_play,field_out,9/16/2022,50
2306,FF,9/16/2022,R,R,0,1,0.0,0.0,0.0,1,...,2,Infield shift,Standard,L,S,FF,called_strike,nothing_major,9/16/2022,51
2307,CH,9/16/2022,R,R,0,2,0.0,0.0,0.0,1,...,2,Infield shift,Standard,B,S,FF,foul,nothing_major,9/16/2022,52
2308,CH,9/16/2022,R,R,1,2,0.0,0.0,0.0,1,...,2,Infield shift,Standard,M,B,CH,ball,nothing_major,9/16/2022,53
2309,FF,9/16/2022,R,R,2,2,0.0,0.0,0.0,1,...,2,Infield shift,Standard,M,B,CH,ball,nothing_major,9/16/2022,54
2310,CU,9/16/2022,L,R,0,0,0.0,0.0,0.0,2,...,2,Infield shift,Standard,C,X,FF,hit_into_play,field_out,9/16/2022,55
2311,FF,9/16/2022,L,R,0,1,0.0,0.0,0.0,2,...,2,Infield shift,Standard,A,S,CU,called_strike,nothing_major,9/16/2022,56
2312,FF,9/16/2022,L,R,1,1,0.0,0.0,0.0,2,...,2,Infield shift,Standard,K,B,FF,ball,nothing_major,9/16/2022,57
2313,FF,9/16/2022,L,R,2,1,0.0,0.0,0.0,2,...,2,Infield shift,Standard,L,B,FF,ball,nothing_major,9/16/2022,58
2314,SL,9/16/2022,L,R,3,1,0.0,0.0,0.0,2,...,2,Infield shift,Standard,L,B,FF,ball,nothing_major,9/16/2022,59


In [12]:
# Converting batting score and fielding score to one column.
# Positive number means fielding team is winning and negative number means batting team is winning. 
verlander_lagged_df['score_diff'] = verlander_lagged_df['fld_score'] - verlander_lagged_df['bat_score']


# Drop batting score and fielding score columns now that you have the score differential
verlander_lagged_df.drop(columns = ['bat_score', 'fld_score', 'p_throws'] , inplace=True)

In [13]:
verlander_lagged_df['balls'] = verlander_lagged_df['balls'].astype(str)
verlander_lagged_df['strikes'] = verlander_lagged_df['strikes'].astype(str)

verlander_lagged_df

,pitch_type,game_date,stand,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,...,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff
0,FF,4/9/2022,L,1,0,0.0,0.0,0.0,0,1,...,Infield shift,Standard,K,B,FF,ball,nothing_major,4/9/2022,1,0
1,FF,4/9/2022,L,1,1,0.0,0.0,0.0,0,1,...,Infield shift,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2,0
2,FF,4/9/2022,L,2,1,0.0,0.0,0.0,0,1,...,Infield shift,Standard,M,B,FF,ball,nothing_major,4/9/2022,3,0
3,FF,4/9/2022,L,2,2,0.0,0.0,0.0,0,1,...,Infield shift,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0
4,FF,4/9/2022,R,0,0,0.0,0.0,0.0,1,1,...,Infield shift,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,SL,9/16/2022,R,0,0,0.0,0.0,0.0,2,5,...,Infield shift,Standard,D,X,SL,hit_into_play,field_out,9/16/2022,75,2
2331,FF,9/16/2022,R,1,0,0.0,0.0,0.0,2,5,...,Infield shift,Standard,H,B,SL,ball,nothing_major,9/16/2022,76,2
2332,SL,9/16/2022,R,1,1,0.0,0.0,0.0,2,5,...,Infield shift,Standard,A,S,FF,foul,nothing_major,9/16/2022,77,2
2333,FF,9/16/2022,R,1,2,0.0,0.0,0.0,2,5,...,Infield shift,Standard,M,S,SL,called_strike,nothing_major,9/16/2022,78,2


In [14]:
verlander_lagged_df['count'] = (verlander_lagged_df['balls'] + verlander_lagged_df['strikes'])
display(verlander_lagged_df)
verlander_lagged_df.isna().sum()

,pitch_type,game_date,stand,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,...,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count
0,FF,4/9/2022,L,1,0,0.0,0.0,0.0,0,1,...,Standard,K,B,FF,ball,nothing_major,4/9/2022,1,0,10
1,FF,4/9/2022,L,1,1,0.0,0.0,0.0,0,1,...,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,11
2,FF,4/9/2022,L,2,1,0.0,0.0,0.0,0,1,...,Standard,M,B,FF,ball,nothing_major,4/9/2022,3,0,21
3,FF,4/9/2022,L,2,2,0.0,0.0,0.0,0,1,...,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,22
4,FF,4/9/2022,R,0,0,0.0,0.0,0.0,1,1,...,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,SL,9/16/2022,R,0,0,0.0,0.0,0.0,2,5,...,Standard,D,X,SL,hit_into_play,field_out,9/16/2022,75,2,00
2331,FF,9/16/2022,R,1,0,0.0,0.0,0.0,2,5,...,Standard,H,B,SL,ball,nothing_major,9/16/2022,76,2,10
2332,SL,9/16/2022,R,1,1,0.0,0.0,0.0,2,5,...,Standard,A,S,FF,foul,nothing_major,9/16/2022,77,2,11
2333,FF,9/16/2022,R,1,2,0.0,0.0,0.0,2,5,...,Standard,M,S,SL,called_strike,nothing_major,9/16/2022,78,2,12


pitch_type               0
game_date                0
stand                    0
balls                    0
strikes                  0
on_3b                    0
on_2b                    0
on_1b                    0
outs_when_up             0
inning                   0
pitch_number             0
if_fielding_alignment    0
of_fielding_alignment    0
previous_zone            0
previous_type            0
previous_pitch_type      0
previous_description     0
previous_event           0
game_date                0
pitch_count              0
score_diff               0
count                    0
dtype: int64

In [15]:
verlander_lagged_df['count']=verlander_lagged_df['count'].map({'30':1,
                                  '31':2,
                                  '20':3,
                                  '32':4,
                                  '21':5,
                                  '10':6,
                                  '00':7,
                                  '11':8,
                                  '01':9,
                                  '22':10,
                                  '12':11,
                                  '02':12   
                                 })
verlander_lagged_df.drop(columns=['balls','strikes'],inplace=True)
verlander_lagged_df

,pitch_type,game_date,stand,on_3b,on_2b,on_1b,outs_when_up,inning,pitch_number,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count
0,FF,4/9/2022,L,0.0,0.0,0.0,0,1,2,Infield shift,Standard,K,B,FF,ball,nothing_major,4/9/2022,1,0,6
1,FF,4/9/2022,L,0.0,0.0,0.0,0,1,3,Infield shift,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,8
2,FF,4/9/2022,L,0.0,0.0,0.0,0,1,4,Infield shift,Standard,M,B,FF,ball,nothing_major,4/9/2022,3,0,5
3,FF,4/9/2022,L,0.0,0.0,0.0,0,1,5,Infield shift,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,10
4,FF,4/9/2022,R,0.0,0.0,0.0,1,1,1,Infield shift,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,SL,9/16/2022,R,0.0,0.0,0.0,2,5,1,Infield shift,Standard,D,X,SL,hit_into_play,field_out,9/16/2022,75,2,7
2331,FF,9/16/2022,R,0.0,0.0,0.0,2,5,2,Infield shift,Standard,H,B,SL,ball,nothing_major,9/16/2022,76,2,6
2332,SL,9/16/2022,R,0.0,0.0,0.0,2,5,3,Infield shift,Standard,A,S,FF,foul,nothing_major,9/16/2022,77,2,8
2333,FF,9/16/2022,R,0.0,0.0,0.0,2,5,4,Infield shift,Standard,M,S,SL,called_strike,nothing_major,9/16/2022,78,2,11


In [16]:
verlander_lagged_df['on_1b'] = verlander_lagged_df['on_1b'].astype('int64').astype(str)
verlander_lagged_df['on_2b'] = verlander_lagged_df['on_2b'].astype('int64').astype(str)
verlander_lagged_df['on_3b'] = verlander_lagged_df['on_3b'].astype('int64').astype(str)
verlander_lagged_df

,pitch_type,game_date,stand,on_3b,on_2b,on_1b,outs_when_up,inning,pitch_number,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count
0,FF,4/9/2022,L,0,0,0,0,1,2,Infield shift,Standard,K,B,FF,ball,nothing_major,4/9/2022,1,0,6
1,FF,4/9/2022,L,0,0,0,0,1,3,Infield shift,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,8
2,FF,4/9/2022,L,0,0,0,0,1,4,Infield shift,Standard,M,B,FF,ball,nothing_major,4/9/2022,3,0,5
3,FF,4/9/2022,L,0,0,0,0,1,5,Infield shift,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,10
4,FF,4/9/2022,R,0,0,0,1,1,1,Infield shift,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,SL,9/16/2022,R,0,0,0,2,5,1,Infield shift,Standard,D,X,SL,hit_into_play,field_out,9/16/2022,75,2,7
2331,FF,9/16/2022,R,0,0,0,2,5,2,Infield shift,Standard,H,B,SL,ball,nothing_major,9/16/2022,76,2,6
2332,SL,9/16/2022,R,0,0,0,2,5,3,Infield shift,Standard,A,S,FF,foul,nothing_major,9/16/2022,77,2,8
2333,FF,9/16/2022,R,0,0,0,2,5,4,Infield shift,Standard,M,S,SL,called_strike,nothing_major,9/16/2022,78,2,11


In [17]:

verlander_lagged_df['bases'] = (verlander_lagged_df['on_3b'] + verlander_lagged_df['on_2b'] + verlander_lagged_df['on_1b'])
display(verlander_lagged_df)

verlander_lagged_df['bases']=verlander_lagged_df['bases'].map({'111':1,
                                  '110':2,
                                  '101':3,
                                  '100':4,
                                  '011':5,
                                  '010':6,
                                  '001':7,  
                                  '000':8
                                 })
verlander_lagged_df.drop(columns=['on_3b','on_2b','on_1b'],inplace=True)
verlander_lagged_df.head(20)

,pitch_type,game_date,stand,on_3b,on_2b,on_1b,outs_when_up,inning,pitch_number,if_fielding_alignment,...,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count,bases
0,FF,4/9/2022,L,0,0,0,0,1,2,Infield shift,...,K,B,FF,ball,nothing_major,4/9/2022,1,0,6,000
1,FF,4/9/2022,L,0,0,0,0,1,3,Infield shift,...,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,8,000
2,FF,4/9/2022,L,0,0,0,0,1,4,Infield shift,...,M,B,FF,ball,nothing_major,4/9/2022,3,0,5,000
3,FF,4/9/2022,L,0,0,0,0,1,5,Infield shift,...,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,10,000
4,FF,4/9/2022,R,0,0,0,1,1,1,Infield shift,...,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,7,000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,SL,9/16/2022,R,0,0,0,2,5,1,Infield shift,...,D,X,SL,hit_into_play,field_out,9/16/2022,75,2,7,000
2331,FF,9/16/2022,R,0,0,0,2,5,2,Infield shift,...,H,B,SL,ball,nothing_major,9/16/2022,76,2,6,000
2332,SL,9/16/2022,R,0,0,0,2,5,3,Infield shift,...,A,S,FF,foul,nothing_major,9/16/2022,77,2,8,000
2333,FF,9/16/2022,R,0,0,0,2,5,4,Infield shift,...,M,S,SL,called_strike,nothing_major,9/16/2022,78,2,11,000


,pitch_type,game_date,stand,outs_when_up,inning,pitch_number,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count,bases
0,FF,4/9/2022,L,0,1,2,Infield shift,Standard,K,B,FF,ball,nothing_major,4/9/2022,1,0,6,8
1,FF,4/9/2022,L,0,1,3,Infield shift,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,8,8
2,FF,4/9/2022,L,0,1,4,Infield shift,Standard,M,B,FF,ball,nothing_major,4/9/2022,3,0,5,8
3,FF,4/9/2022,L,0,1,5,Infield shift,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,10,8
4,FF,4/9/2022,R,1,1,1,Infield shift,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,7,8
5,FF,4/9/2022,R,1,1,2,Infield shift,Standard,L,B,FF,ball,nothing_major,4/9/2022,6,0,6,8
6,FF,4/9/2022,R,1,1,3,Infield shift,Standard,H,S,FF,foul,nothing_major,4/9/2022,7,0,8,8
7,SL,4/9/2022,R,1,1,4,Infield shift,Standard,C,S,FF,foul,nothing_major,4/9/2022,8,0,11,8
8,FF,4/9/2022,R,1,1,5,Infield shift,Standard,N,B,SL,ball,nothing_major,4/9/2022,9,0,10,8
9,FF,4/9/2022,R,2,1,1,Standard,Standard,B,X,FF,hit_into_play,field_out,4/9/2022,10,0,7,8


In [18]:
# verlander_df.on_3b[verlander_df.on_3b > 0] = 1.0

verlander_lagged_df.loc[verlander_lagged_df['pitch_number'] > 1, 'pitch_number'] = 0
verlander_lagged_df

,pitch_type,game_date,stand,outs_when_up,inning,pitch_number,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count,bases
0,FF,4/9/2022,L,0,1,0,Infield shift,Standard,K,B,FF,ball,nothing_major,4/9/2022,1,0,6,8
1,FF,4/9/2022,L,0,1,0,Infield shift,Standard,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,8,8
2,FF,4/9/2022,L,0,1,0,Infield shift,Standard,M,B,FF,ball,nothing_major,4/9/2022,3,0,5,8
3,FF,4/9/2022,L,0,1,0,Infield shift,Standard,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,10,8
4,FF,4/9/2022,R,1,1,1,Infield shift,Standard,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,SL,9/16/2022,R,2,5,1,Infield shift,Standard,D,X,SL,hit_into_play,field_out,9/16/2022,75,2,7,8
2331,FF,9/16/2022,R,2,5,0,Infield shift,Standard,H,B,SL,ball,nothing_major,9/16/2022,76,2,6,8
2332,SL,9/16/2022,R,2,5,0,Infield shift,Standard,A,S,FF,foul,nothing_major,9/16/2022,77,2,8,8
2333,FF,9/16/2022,R,2,5,0,Infield shift,Standard,M,S,SL,called_strike,nothing_major,9/16/2022,78,2,11,8


In [19]:

verlander_lagged_df.rename(columns = {
    'pitch_number' : '1st_pitch',
}, inplace=True)



verlander_lagged_df.tail(30)


,pitch_type,game_date,stand,outs_when_up,inning,1st_pitch,if_fielding_alignment,of_fielding_alignment,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count,bases
2305,FF,9/16/2022,R,1,4,1,Infield shift,Standard,F,X,SL,hit_into_play,field_out,9/16/2022,50,2,7,8
2306,FF,9/16/2022,R,1,4,0,Infield shift,Standard,L,S,FF,called_strike,nothing_major,9/16/2022,51,2,9,8
2307,CH,9/16/2022,R,1,4,0,Infield shift,Standard,B,S,FF,foul,nothing_major,9/16/2022,52,2,12,8
2308,CH,9/16/2022,R,1,4,0,Infield shift,Standard,M,B,CH,ball,nothing_major,9/16/2022,53,2,11,8
2309,FF,9/16/2022,R,1,4,0,Infield shift,Standard,M,B,CH,ball,nothing_major,9/16/2022,54,2,10,8
2310,CU,9/16/2022,L,2,4,1,Infield shift,Standard,C,X,FF,hit_into_play,field_out,9/16/2022,55,2,7,8
2311,FF,9/16/2022,L,2,4,0,Infield shift,Standard,A,S,CU,called_strike,nothing_major,9/16/2022,56,2,9,8
2312,FF,9/16/2022,L,2,4,0,Infield shift,Standard,K,B,FF,ball,nothing_major,9/16/2022,57,2,8,8
2313,FF,9/16/2022,L,2,4,0,Infield shift,Standard,L,B,FF,ball,nothing_major,9/16/2022,58,2,5,8
2314,SL,9/16/2022,L,2,4,0,Infield shift,Standard,L,B,FF,ball,nothing_major,9/16/2022,59,2,2,8


In [20]:
verlander_lagged_df['inning'] = verlander_lagged_df['inning'].astype(str)

In [21]:
verlander_lagged_df = verlander_lagged_df.drop(columns=[
                                                        # 'previous_zone',
                                                        # 'previous_description',
                                                        # 'previous_event',
                                                        'if_fielding_alignment',
                                                        'of_fielding_alignment',
                                                        'inning',
                                                        # '1st_pitch'
                                                       ]
                                              )

In [22]:

display(verlander_lagged_df.head())

# Split target columns from features
y = verlander_lagged_df['pitch_type']
X = verlander_lagged_df.drop(columns = ['pitch_type','game_date'])
X.dtypes

,pitch_type,game_date,stand,outs_when_up,1st_pitch,previous_zone,previous_type,previous_pitch_type,previous_description,previous_event,game_date,pitch_count,score_diff,count,bases
0,FF,4/9/2022,L,0,0,K,B,FF,ball,nothing_major,4/9/2022,1,0,6,8
1,FF,4/9/2022,L,0,0,M,S,FF,called_strike,nothing_major,4/9/2022,2,0,8,8
2,FF,4/9/2022,L,0,0,M,B,FF,ball,nothing_major,4/9/2022,3,0,5,8
3,FF,4/9/2022,L,0,0,D,S,FF,foul_tip,nothing_major,4/9/2022,4,0,10,8
4,FF,4/9/2022,R,1,1,H,S,FF,swinging_strike,strikeout,4/9/2022,5,0,7,8


stand                   object
outs_when_up             int64
1st_pitch                int64
previous_zone           object
previous_type           object
previous_pitch_type     object
previous_description    object
previous_event          object
pitch_count              int64
score_diff               int64
count                    int64
bases                    int64
dtype: object

In [23]:
# Create a list of categorical variables
categorical_variables = list(X.dtypes[X.dtypes == 'object'].index)

# Display the categorical variables list
categorical_variables

['stand',
 'previous_zone',
 'previous_type',
 'previous_pitch_type',
 'previous_description',
 'previous_event']

In [24]:
# display_full(X)
# display_full(X_encoded_df)

In [25]:
# display_full(encoded_df)
# display_full(numerical_variables)

In [26]:
# Change categorical variables using OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse = False)

encoded_data = enc.fit_transform(X[categorical_variables])
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

numerical_variables = X.drop(columns = categorical_variables)
X_encoded_df = pd.concat([encoded_df, numerical_variables], axis = 1)

def display_full(x):
    pd.set_option('display.max_columns', 100)
    display(x)
    pd.reset_option('display.max_columns')
    
display_full(X_encoded_df.head())

C:\anacondastuff\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,stand_L,stand_R,previous_zone_A,previous_zone_B,previous_zone_C,previous_zone_D,previous_zone_E,previous_zone_F,previous_zone_G,previous_zone_H,previous_zone_I,previous_zone_K,previous_zone_L,previous_zone_M,previous_zone_N,previous_type_B,previous_type_S,previous_type_X,previous_pitch_type_CH,previous_pitch_type_CU,previous_pitch_type_FF,previous_pitch_type_SL,previous_description_ball,previous_description_blocked_ball,previous_description_called_strike,previous_description_foul,previous_description_foul_bunt,previous_description_foul_tip,previous_description_hit_by_pitch,previous_description_hit_into_play,previous_description_swinging_strike,previous_description_swinging_strike_blocked,previous_event_caught_stealing_2b,previous_event_double,previous_event_double_play,previous_event_field_error,previous_event_field_out,previous_event_force_out,previous_event_grounded_into_double_play,previous_event_hit_by_pitch,previous_event_home_run,previous_event_nothing_major,previous_event_other_out,previous_event_sac_bunt,previous_event_sac_fly,previous_event_single,previous_event_strikeout,previous_event_triple,previous_event_walk,outs_when_up,1st_pitch,pitch_count,score_diff,count,bases
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,6,8
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2,0,8,8
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3,0,5,8
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4,0,10,8
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,5,0,7,8


In [27]:
X_encoded_df.drop(columns='stand_L',inplace=True)
X_encoded_df

,stand_R,previous_zone_A,previous_zone_B,previous_zone_C,previous_zone_D,previous_zone_E,previous_zone_F,previous_zone_G,previous_zone_H,previous_zone_I,...,previous_event_single,previous_event_strikeout,previous_event_triple,previous_event_walk,outs_when_up,1st_pitch,pitch_count,score_diff,count,bases
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,6,8
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,2,0,8,8
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,3,0,5,8
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,4,0,10,8
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1,1,5,0,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,1,75,2,7,8
2331,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2,0,76,2,6,8
2332,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,0,77,2,8,8
2333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,0,78,2,11,8


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, random_state=1)

# Scaling Data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [29]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf_model = rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)

# Displaying results
print(classification_report(y_test, rf_predictions))

# Zip the feature importances with the associated feature name
importances = rf_model.feature_importances_
important_features = zip(X_encoded_df.columns, importances)
important_features
# Calculating the most important features that the Random Forest model used
importances_df = pd.DataFrame(important_features)
importances_df = importances_df.rename(columns = {0: 'Feature', 1: 'Importance'})
importances_df = importances_df.set_index('Feature')
importances_df = importances_df.sort_values(by = 'Importance', ascending = False)
importances_df

              precision    recall  f1-score   support

          CH       0.00      0.00      0.00        11
          CU       0.39      0.28      0.33        95
          FF       0.62      0.78      0.69       305
          SL       0.54      0.41      0.47       173

    accuracy                           0.58       584
   macro avg       0.39      0.37      0.37       584
weighted avg       0.55      0.58      0.55       584



,Importance
Feature,
pitch_count,0.219440
score_diff,0.113981
count,0.110624
outs_when_up,0.071667
bases,0.064418
stand_R,0.039137
previous_pitch_type_FF,0.027146
previous_pitch_type_SL,0.021023
previous_pitch_type_CU,0.019188


In [30]:
# display_full(importances_df)